 # Simulating CH4 with PennyLane 

## Building molecular Hamiltonian 

In [ ]:
from pennylane import numpy as np 
from pennylane import qchem 
import pennylane as qml 
import time 

# defining the molecule 

symbols = ["C", "H", "H", "H", "H"]
coordinates = 0.529*np.array([[0.0,0.0,0.0],[0.6276,0.6276,0.6276],[0.6276,-0.6276,-0.6276],[-0.6276,0.6276,-0.6276],[-0.6276,-0.6276,0.6276]])
# coordinates in atomic units 

In [ ]:
# defining molecule specificites for simplification after having studied it's MOs

charge = 0  # not an ion 
mult = 1 # initially, prone to change if Givens rotations 
active_electrons = 8 # not considering the 1s core electrons 
active_orbitals = 5 # neglecting core 1s and the 3 MOs with the highest energy

H, qubits = qchem.molecular_hamiltonian(symbols,coordinates,charge=charge, mult = mult, active_electrons = active_electrons, active_orbitals = active_orbitals)


## Energy and ground state 

In [ ]:
electrons = 8
hf = qml.qchem.hf_state(electrons, qubits) # creating corresponding Hartree-Fock state
print(hf)

### Evaluating the relevant Givens rotations (i.e. electrons excitations)

In [ ]:
singles, doubles = qchem.excitations(active_electrons,qubits)
print("Total number of excitations = {}".format(len(singles)+len(doubles))) # yields 24 possible spin-preserving excitations

def circuit_1(params, excitations): 
    qml.BasisState(hf_state,wires=range(qubits)) # generating the corresponding Hartree-Fock state in the the VQC
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4: # meaning a double-excitation 
            qml.DoubleExcitation(params[i],wires=excitation) # where the excitation is being applied
        else: # meaning single-excitation 
            qml.SingleExcitation(params[i],wires=excitation) 
    return qml.expval(H)